# Smart Cultural Storyteller - AI Content Generation

This notebook generates images and videos for the Betal storytelling project using free AI models.

## Setup and Dependencies

In [3]:
# Install required packages
!pip install requests pillow opencv-python moviepy huggingface-hub transformers torch diffusers
!pip install elevenlabs gTTS  # For text-to-speech
!pip install gradio  # For easy UI testing


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\itske\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: '#'

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\itske\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: '#'

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\itske\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
import requests
import json
import os
import time
from PIL import Image, ImageEnhance, ImageFilter
import cv2
import numpy as np
from moviepy.editor import *
#import torch
#from diffusers import StableDiffusionPipeline
from gtts import gTTS
import io
from pathlib import Path

## Configuration

In [ ]:
# API Keys - Replace with your actual keys
HUGGINGFACE_TOKEN = os.getenv("HUGGINGFACE_TOKEN")
ELEVENLABS_API_KEY = os.getenv("ELEVENLABS_API_KEY")

# Output directories
OUTPUT_DIR = Path("generated_content")
IMAGES_DIR = OUTPUT_DIR / "images"
AUDIO_DIR = OUTPUT_DIR / "audio"
VIDEOS_DIR = OUTPUT_DIR / "videos"

# Create directories
for dir_path in [IMAGES_DIR, AUDIO_DIR, VIDEOS_DIR]:
    dir_path.mkdir(parents=True, exist_ok=True)

## Story Data

In [7]:
# Sample story for generation
story_data = {
    "id": "wisdom-1",
    "title": "The Wise Old Man and the Three Sons",
    "content": """Once upon a time, in a small village, there lived an old man with three sons. 
    The old man was known throughout the village for his wisdom and kindness. 
    As he grew older, he wanted to test which of his sons would inherit his wisdom.
    He gave each son a single grain of rice and told them to make it multiply by the next full moon.
    The first son planted it and got a small harvest. 
    The second son sold it and bought more rice. 
    But the third son gave it to a hungry child, saying that kindness multiplies in ways grain cannot.""",
    "scenes": [
        "An old wise man in traditional Indian clothing in a village setting",
        "Three sons receiving grains of rice from their father",
        "First son planting rice in a field, traditional Indian farming",
        "Second son at a marketplace exchanging rice for money", 
        "Third son giving rice to a hungry child, showing compassion",
        "The wise old man smiling, understanding true wisdom"
    ]
}

## Image Generation using Stable Diffusion

In [39]:
def check_model_status(model_name):
    """Check if the model is loaded and ready"""
    API_URL = f"https://api-inference.huggingface.co/models/{model_name}"
    headers = {"Authorization": f"Bearer {HUGGINGFACE_TOKEN}"}
    
    response = requests.get(API_URL, headers=headers)
    print(f"Model status: {response.status_code}")
    if response.status_code == 200:
        return True
    return False

def generate_image_hf_api(prompt, filename, max_retries=3):
    """Generate image using Hugging Face Inference API with better error handling"""
    
    # Try different models if one fails
    models = [
        "black-forest-labs/FLUX.1-schnell"
    ]
    
    for model in models:
        API_URL = f"https://api-inference.huggingface.co/models/{model}"
        headers = {"Authorization": f"Bearer {HUGGINGFACE_TOKEN}"}
        
        # Enhanced prompt for Indian cultural context
        enhanced_prompt = f"{prompt}, Indian cultural motifs, traditional art style, highly detailed, rich textures, vivid harmonious colors, intricate patterns, cinematic soft lighting, emotional storytelling, high quality"
        
        payload = {
            "inputs": enhanced_prompt,
            "parameters": {
                "guidance_scale": 7.5,
                "num_inference_steps": 50,
                "width": 512,
                "height": 512
            }
        }
        
        for attempt in range(max_retries):
            try:
                print(f"Attempting to generate: {prompt[:50]}... (Model: {model}, Attempt: {attempt+1})")
                
                response = requests.post(API_URL, headers=headers, json=payload, timeout=60)
                
                if response.status_code == 200:
                    # Check if response is actually an image
                    if response.headers.get('content-type', '').startswith('image/'):
                        image_path = IMAGES_DIR / f"{filename}.png"
                        with open(image_path, "wb") as f:
                            f.write(response.content)
                        print(f"✅ Generated image: {image_path}")
                        return image_path
                    else:
                        print(f"❌ Response is not an image. Content-Type: {response.headers.get('content-type')}")
                        try:
                            error_data = response.json()
                            print(f"Error response: {error_data}")
                        except:
                            print(f"Raw response: {response.text[:200]}...")
                
                elif response.status_code == 503:
                    try:
                        error_data = response.json()
                        if "loading" in str(error_data).lower():
                            estimated_time = error_data.get("estimated_time", 30)
                            print(f"⏳ Model is loading. Waiting {estimated_time} seconds...")
                            time.sleep(estimated_time + 10)  # Wait a bit longer
                            continue
                    except:
                        pass
                    print(f"❌ Service unavailable: {response.text}")
                
                else:
                    print(f"❌ Error {response.status_code}: {response.text}")
                
                # Wait before retry
                if attempt < max_retries - 1:
                    wait_time = (attempt + 1) * 5
                    print(f"⏳ Waiting {wait_time} seconds before retry...")
                    time.sleep(wait_time)
                    
            except requests.exceptions.Timeout:
                print(f"⏳ Request timeout. Retrying...")
                time.sleep(10)
            except Exception as e:
                print(f"❌ Exception: {str(e)}")
                time.sleep(5)
        
        print(f"❌ Failed with model {model}, trying next model...")
        time.sleep(5)
    
    print("❌ All models failed")
    return None

def test_api_connection():
    """Test the API connection and token validity"""
    headers = {"Authorization": f"Bearer {HUGGINGFACE_TOKEN}"}
    test_url = "https://api-inference.huggingface.co/models/runwayml/stable-diffusion-v1-5"
    
    try:
        response = requests.get(test_url, headers=headers)
        print(f"API Test - Status Code: {response.status_code}")
        if response.status_code == 401:
            print("❌ Invalid or missing token. Please check your HUGGINGFACE_TOKEN")
            return False
        elif response.status_code == 200:
            print("✅ API connection successful")
            return True
        else:
            print(f"⚠️ Unexpected response: {response.text}")
            return True  # Might still work
    except Exception as e:
        print(f"❌ API test failed: {str(e)}")
        return False

In [ ]:
def generate_story_images(story_data):
    """Generate images for the story with better error handling"""
    
    # Test API connection first
    if not test_api_connection():
        print("❌ API connection test failed. Please check your token.")
        return []
    
    image_paths = []
    
    for i, scene_prompt in enumerate(story_data["scenes"]):
        print(f"\n🎨 Generating image {i+1}/{len(story_data['scenes'])}")
        filename = f"{story_data['id']}_scene_{i+1:02d}"
        image_path = generate_image_hf_api(scene_prompt, filename)
        
        if image_path:
            image_paths.append(str(image_path))
        else:
            print(f"❌ Failed to generate image for scene {i+1}")
        
        # Add delay to avoid rate limiting
        print("⏳ Waiting to avoid rate limiting...")
        time.sleep(10)  # Increased delay
    
    return image_paths

def test_single_image():
    """Test with a single image generation"""
    print("🧪 Testing single image generation...")
    test_prompt = "A beautiful traditional Indian village scene with houses and trees"
    result = generate_image_hf_api(test_prompt, "test_image")
    if result:
        print("✅ Single image test successful!")
        return True
    else:
        print("❌ Single image test failed")
        return False

# Main execution - FIXED SYNTAX
if __name__ == "__main__":  # Changed from **name** to __name__
    print("🚀 Starting image generation process...")
    
    # First, test with a single image
    if test_single_image():
        print("\n🎨 Generating story images...")
        image_paths = generate_story_images(story_data)
        print(f"\n✅ Successfully generated {len(image_paths)} images out of {len(story_data['scenes'])} scenes")
        
        # Display generated images info
        for i, path in enumerate(image_paths):
            if os.path.exists(path):
                file_size = os.path.getsize(path) / 1024  # KB
                print(f"  📸 Scene {i+1}: {path} ({file_size:.1f} KB)")
            else:
                print(f"  ❌ Scene {i+1}: File not found at {path}")
    else:
        print("❌ Skipping full generation due to test failure")

# Alternative: Run without the main check (for Jupyter/Colab)
def run_image_generation():
    """Alternative function to run in Jupyter/Colab environments"""
    print("🚀 Starting image generation process...")
    
    # First, test with a single image
    if test_single_image():
        print("\n🎨 Generating story images...")
        image_paths = generate_story_images(story_data)
        print(f"\n✅ Successfully generated {len(image_paths)} images out of {len(story_data['scenes'])} scenes")
        
        # Display generated images info
        for i, path in enumerate(image_paths):
            if os.path.exists(path):
                file_size = os.path.getsize(path) / 1024  # KB
                print(f"  📸 Scene {i+1}: {path} ({file_size:.1f} KB)")
            else:
                print(f"  ❌ Scene {i+1}: File not found at {path}")
        
        return image_paths
    else:
        print("❌ Skipping full generation due to test failure")
        return []

# If you're running in Jupyter/Colab, uncomment the next line:
# image_paths = run_image_generation()

🚀 Starting image generation process...
🧪 Testing single image generation...
Attempting to generate: A beautiful traditional Indian village scene with ... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)


KeyboardInterrupt: 

## Audio Generation (Text-to-Speech)

In [41]:
# Option 1: Use gTTS (Free, supports Indian English accent)
def generate_audio_gtts(text, filename, lang='en', tld='co.in'):
    """Generate audio using Google Text-to-Speech with Indian accent (male-sounding)"""
    tts = gTTS(text=text, lang=lang, tld=tld, slow=True)  # Slower speech for deeper tone
    audio_path = AUDIO_DIR / f"{filename}.mp3"
    tts.save(str(audio_path))
    print(f"Generated audio: {audio_path}")
    return audio_path



In [13]:
# Generate narration audio
narration_text = story_data["content"].replace('\n', ' ').strip()
audio_filename = f"{story_data['id']}_narration"

print("Generating narration audio...")
# Use gTTS for free option
audio_path = generate_audio_gtts(narration_text, audio_filename)

# Uncomment below to use ElevenLabs instead
# audio_path = generate_audio_elevenlabs(narration_text, audio_filename)

Generating narration audio...
Generated audio: generated_content\audio\wisdom-1_narration.mp3


## Video Generation with Effects

In [42]:
from pathlib import Path
import os, re
import numpy as np
from moviepy.editor import (
    ImageClip,
    CompositeVideoClip,
    concatenate_videoclips,
    AudioFileClip,
)

def _safe_name(name: str) -> str:
    base = os.path.splitext(os.path.basename(str(name)))[0]
    return re.sub(r'[^A-Za-z0-9._-]+', '_', base)[:80]

def ease_in_out_cubic(x: float) -> float:
    return 4 * x * x * x if x < 0.5 else 1 - (-2 * x + 2) ** 3 / 2

import cv2
import numpy as np
import moviepy.editor as mp
def get_blurred_background_cv2(img_path, out_size, blur_ksize=51):
    img = cv2.imread(img_path)
    img = cv2.resize(img, out_size)
    img = cv2.GaussianBlur(img, (blur_ksize, blur_ksize), 0)
    tmp_path = "temp_blur.jpg"
    cv2.imwrite(tmp_path, img)
    return mp.ImageClip(tmp_path).set_duration(5)


def add_parallax_effect(image_path, duration=5, out_size=(1280, 720), variant_idx=0, zoom=0.04, pan_px=30):
    from moviepy.editor import ImageClip, CompositeVideoClip, vfx
    import numpy as np

    def ease_in_out_cubic(x):
        return 4*x*x*x if x < 0.5 else 1 - (-2*x + 2) ** 3 / 2

    base = ImageClip(image_path).set_duration(duration)
    W, H = out_size
    img_w, img_h = base.size

    # scale image to fit inside frame (letterbox style)
    fit_scale = min(W / img_w, H / img_h)
    scale_start = fit_scale
    scale_end   = fit_scale * (1.0 + max(0.0, zoom))

    def scale_fn(t):
        p = ease_in_out_cubic(t / duration)
        return scale_start + (scale_end - scale_start) * p

    rng = np.random.RandomState(variant_idx or 0)
    pan_x = int(rng.choice([-1, 1]) * rng.randint(max(12, pan_px // 2), pan_px))
    pan_y = int(rng.choice([-1, 1]) * rng.randint(max(8, pan_px // 3), max(14, pan_px)))

    def pos_fn(t):
        p = ease_in_out_cubic(t / duration)
        # center alignment + parallax pan
        return (
            (W - img_w * scale_fn(t)) / 2 - pan_x * (1 - p),
            (H - img_h * scale_fn(t)) / 2 - pan_y * (1 - p),
        )

    # Foreground (main image, centered and animated)
    moving = base.resize(lambda t: scale_fn(t))

    # Background (blurred, scaled to cover full frame)
    background = get_blurred_background_cv2(image_path, out_size)


    return CompositeVideoClip(
        [background, moving.set_position(pos_fn)],
        size=out_size
    ).set_duration(duration)


def create_story_video(
    image_paths,
    audio_path,
    story_id,
    out_size=(1280, 720),
    fps=24,
    crossfade=0.3,       # set 0.0 to disable zoom entirely        # set 0 to disable pan entirely
    videos_dir=None,
):
    # Output directory
    if videos_dir is None:
        try:
            output_dir = Path(VIDEOS_DIR)
        except NameError:
            output_dir = Path("generated_content") / "videos"
    else:
        output_dir = Path(videos_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    # Audio
    if not (audio_path and os.path.exists(str(audio_path))):
        print("Audio file missing; provide valid audio_path.")
        return None
    audio_clip = AudioFileClip(str(audio_path))
    total_duration = audio_clip.duration

    n = len(image_paths)
    if n == 0:
        print("No images provided.")
        return None

    per = total_duration / n

    clips = []
    for i, img_path in enumerate(image_paths):
        if not os.path.exists(img_path):
            print(f"Image not found: {img_path}")
            continue

        start = i * per
        dur = per if i < n - 1 else max(0.1, total_duration - start)

        clip = add_parallax_effect(
            img_path,
            duration=dur,
            out_size=out_size,
            variant_idx=i
        )
        clip = clip.set_start(start)
        if i > 0 and crossfade > 0:
            clip = clip.crossfadein(min(crossfade, dur / 2))
        clips.append(clip)

    if not clips:
        print("No valid clips to compose.")
        return None

    timeline = concatenate_videoclips(clips, method="compose", padding=0)
    timeline = timeline.set_audio(audio_clip).set_fps(fps)

    safe_id = _safe_name(story_id)
    video_path = output_dir / f"{safe_id}_video.mp4"
    timeline.write_videofile(
        str(video_path),
        codec="libx264",
        audio=True,
        audio_codec="aac",
        fps=fps,
        remove_temp=True,
    )
    return str(video_path)

In [35]:
# Create the story video
print("Creating story video with effects...")
video_path = create_story_video(
    image_paths=image_paths,
    audio_path=audio_path,
    story_id=story_data["id"],
    out_size=(1280, 720),
    fps=24,
    crossfade=0.3   # try 0.05–0.06 if still subtle  # try 36–48 if still subtle
)
if video_path:
    print(f"\nStory video created successfully: {video_path}")
    print(f"You can now use this video in your web application!")
else:
    print("Failed to create video")

Creating story video with effects...
Moviepy - Building video generated_content\videos\wisdom-1_video.mp4.
MoviePy - Writing audio in wisdom-1_videoTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video generated_content\videos\wisdom-1_video.mp4



Moviepy - Done !
Moviepy - video ready generated_content\videos\wisdom-1_video.mp4

Story video created successfully: generated_content\videos\wisdom-1_video.mp4
You can now use this video in your web application!


## Generate Subtitles

In [43]:
def generate_subtitles(story_content, audio_duration):
    """Generate subtitle timing for the story"""
    sentences = story_content.replace('\n', ' ').split('. ')
    sentences = [s.strip() + '.' for s in sentences if s.strip()]
    
    duration_per_sentence = audio_duration / len(sentences)
    
    subtitles = []
    current_time = 0
    
    for sentence in sentences:
        subtitle = {
            "start": round(current_time, 1),
            "end": round(current_time + duration_per_sentence, 1),
            "text": sentence
        }
        subtitles.append(subtitle)
        current_time += duration_per_sentence
    
    return subtitles

# Generate subtitles
from moviepy.editor import AudioFileClip
audio_clip = AudioFileClip(str(audio_path))
subtitles = generate_subtitles(story_data['content'], audio_clip.duration)

print("Generated subtitles:")
for subtitle in subtitles:
    print(f"{subtitle['start']}s - {subtitle['end']}s: {subtitle['text']}")

Generated subtitles:
0s - 6.4s: Once upon a time, in a small village, there lived an old man with three sons.
6.4s - 12.9s: The old man was known throughout the village for his wisdom and kindness.
12.9s - 19.3s: As he grew older, he wanted to test which of his sons would inherit his wisdom.
19.3s - 25.8s: He gave each son a single grain of rice and told them to make it multiply by the next full moon.
25.8s - 32.2s: The first son planted it and got a small harvest.
32.2s - 38.7s: The second son sold it and bought more rice.
38.7s - 45.1s: But the third son gave it to a hungry child, saying that kindness multiplies in ways grain cannot..


## Export Data for Web Application

In [21]:
# Create JSON data for web application
web_story_data = {
    "id": story_data['id'],
    "title": story_data['title'],
    "content": story_data['content'],
    "images": [f"/generated_content/images/{Path(p).name}" for p in image_paths],
    "audioUrl": f"/generated_content/audio/{audio_path.name}",
    "videoUrl": f"/generated_content/videos/{video_path}" if video_path else None,
    "subtitles": subtitles
}

# Save to JSON file
output_json = OUTPUT_DIR / f"{story_data['id']}_data.json"
with open(output_json, 'w') as f:
    json.dump(web_story_data, f, indent=2)

print(f"\nWeb application data saved to: {output_json}")
print("\nGenerated files:")
print(f"- Images: {len(image_paths)} files in {IMAGES_DIR}")
print(f"- Audio: {audio_path}")
if video_path:
    print(f"- Video: {video_path}")
print(f"- Data: {output_json}")


Web application data saved to: generated_content\wisdom-1_data.json

Generated files:
- Images: 6 files in generated_content\images
- Audio: generated_content\audio\wisdom-1_narration.mp3
- Video: generated_content\videos\wisdom-1_video.mp4
- Data: generated_content\wisdom-1_data.json


## Batch Process All Stories

In [44]:
# Function to process all 10 stories
def process_all_stories():
    """Process all stories in the dataset"""
    
    # You can define all 10 stories here or load from external file
    stories = stories = [
  {
    "id": "wisdom-1",
    "title": "The Wise Old Man and the Three Sons",
    "theme": "wisdom",
    "content": "Once upon a time, in a small village, there lived an old man with three sons. The old man was known throughout the village for his wisdom and kindness. As he grew older, he wanted to test which of his sons would inherit his wisdom. He gave each son a single grain of rice and told them to make it multiply by the next full moon. The first son planted it and got a small harvest. The second son sold it and bought more rice. But the third son gave it to a hungry child, saying that kindness multiplies in ways grain cannot.",
    "scenes": [
      "Golden late-afternoon courtyard: an elderly man in a worn shawl sits on a charpai handing a single grain of rice to each son — close, warm expressions, mud-brick houses and laced shadows behind him",
      "Morning in the paddy: the first son kneeling in saturated green terraces, hands in wet soil planting a single grain, dew on leaves, focused intimate portrait",
      "Bustling bazaar: the second son at a crowded market stall exchanging coins for sacks of rice, colorful fabrics, dynamic crowd, lens slightly above for cinematic tension",
      "Compassionate act: the third son bending to give his grain to a hungry child beneath a banyan tree, soft golden rim-light, gentle faces, emotional close-up",
      "Sunset resolution: the wise old man gazing proudly from his doorway as his sons stand before him, warm palette, balanced composition, subtle smile"
    ]
  },
  {
    "id": "courage-1",
    "title": "The Little Sparrow and the Storm",
    "theme": "courage",
    "content": "A little sparrow lived in a great banyan tree near a village. When a terrible storm threatened to destroy the village, all the bigger birds flew away to safety. But the little sparrow noticed an old woman who couldn't flee. Despite being tiny, the sparrow decided to help. Through the night, it flew back and forth, warning villagers and guiding them to safety. By morning, the sparrow had saved many lives, proving that courage isn't about size—it's about the size of your heart.",
    "scenes": [
      "Wide majestic banyan: a tiny sparrow perched high in tangled roots and branches, strong scale contrast between bird and tree, early morning mist",
      "Ominous sky: thick rolling storm clouds gathering above the village, wind-blown palms and dark silhouettes, cinematic tension",
      "Panic and flight: villagers scrambling while large birds depart in silhouetted flocks, motion blur and dust rising, chaotic energy",
      "Nighttime heroics: a small sparrow carrying a faint glow, darting between frightened villagers to warn and guide them — dramatic backlit rain and focused emotion",
      "Dawn calm: battered but saved villagers gather by the banyan at sunrise, pale pastel sky, relieved faces, the sparrow resting contentedly"
    ]
  },
  {
    "id": "kindness-1",
    "title": "The Magic Pot of the Poor Woman",
    "theme": "kindness",
    "content": "In a poor village lived a woman with nothing but a small clay pot. One day, a hungry traveler knocked on her door. Though she had only a handful of rice, she shared half with him. The traveler revealed he was a sage and blessed her pot. From that day, the pot never emptied—whatever she put in multiplied. But the woman never kept it all for herself. She fed every hungry person in the village, and her kindness fed hundreds.",
    "scenes": [
      "Humble hearth: close interior of a mud-walled hut with a small fire, the poor woman stirring a tiny pot, warm low-key lighting, detailed textures of clay and woven mat",
      "Knock at dusk: a travel-worn stranger in faded cloak pauses at a rough wooden door, moonlight halo, compassionate exchange about to begin",
      "Last handful shared: extreme close-up of rough hands offering a few grains of rice into another’s palm, soft focus background, moral weight in the gesture",
      "Blessing moment: the sage placing hands over the small clay pot as a faint magical glow rises, dust motes in the light, reverent framing",
      "Communal feast: an overflowing pot in the courtyard with villagers gathered, joyful expressions and vibrant cloths, overhead festival-like composition"
    ]
  },
  {
    "id": "justice-1",
    "title": "The Honest Merchant's Scale",
    "theme": "justice",
    "content": "A merchant in the marketplace was known for his honest scales while others cheated customers. When a drought came, dishonest merchants hoarded grain to sell at high prices. The honest merchant sold his grain at fair prices, even when he could have made more profit. Soon, his was the only shop people trusted. When the rains returned and grain became plentiful again, everyone remembered his fairness. His business prospered while the dishonest merchants lost their customers forever.",
    "scenes": [
      "Market panorama: a vibrant traditional marketplace with colorful awnings and sacks of grain, sunlit dust, busy vendors and bargaining villagers",
      "Portrait of integrity: the honest merchant carefully balancing an old brass scale, focused hands, weathered face, respectful customers in soft focus",
      "Shadowy hoarders: other merchants stacking and hiding sacks in dim corners during drought, stark contrast in lighting to emphasize greed",
      "Trust rewarded: villagers happily buying measured grain at fair prices, candid smiles and exchanged coins, midday clarity",
      "Harvest renewal: heavy rain returning over open fields, lush green beyond the market and the honest merchant's bustling shop full of loyal patrons"
    ]
  },
  {
    "id": "wisdom-2",
    "title": "The Student and the Empty Cup",
    "theme": "wisdom",
    "content": "A proud student visited a wise teacher, boasting about his knowledge. The teacher invited him for tea and began pouring into the student's cup. He kept pouring even as the cup overflowed, spilling tea everywhere. 'Stop!' cried the student. 'The cup is full!' The teacher smiled and said, 'Yes, and like this cup, you are full of your own ideas. How can I teach you anything until you first empty your cup?'",
    "scenes": [
      "Courtyard arrival: a confident young student in neat but showy clothes stands at the entrance of a calm teacher's courtyard, posture rigid, midday light",
      "Tea lesson in progress: the teacher pours steadily as tea flows over an ornate cup, slow-motion feel, liquid catching light, expressions of surprise",
      "Overflow detail: tea spilling across a carved wooden table into patterned inlay, tea droplets frozen in motion, visual metaphor for excess",
      "Moment of realization: the student's face shifting from pride to humility while the teacher remains serene, soft backlight emphasizing the lesson",
      "Quiet closure: teacher smiling gently as the humbled student listens, warm evening tones, balanced composition that underscores transformation"
    ]
  }
]


    
    processed_stories = []
    
    for story in stories:
        print(f"\n=== Processing Story: {story['title']} ===")
        
        try:
            # Generate images
            image_paths = generate_story_images(story)
            
            # Generate audio
            narration_text = story["content"].replace('\n', ' ').strip()
            audio_path = generate_audio_gtts(narration_text, f"{story['id']}_narration")
            
            # Generate video
            video_path = create_story_video(image_paths, audio_path, story['id'])
            
            # Generate subtitles
            audio_clip = AudioFileClip(str(audio_path))
            subtitles = generate_subtitles(story['content'], audio_clip.duration)
            
            # Create web data
            web_data = {
                "id": story['id'],
                "title": story['title'],
                "content": story['content'],
                "images": [f"/generated_content/images/{Path(p).name}" for p in image_paths],
                "audioUrl": f"/generated_content/audio/{audio_path.name}",
                "videoUrl": f"/generated_content/videos/{video_path}" if video_path else None,
                "subtitles": subtitles
            }
            
            processed_stories.append(web_data)
            print(f"✅ Successfully processed: {story['title']}")
            
        except Exception as e:
            print(f"❌ Error processing {story['title']}: {str(e)}")
    
    # Save all processed stories
    all_stories_file = OUTPUT_DIR / "all_stories_data.json"
    with open(all_stories_file, 'w') as f:
        json.dump(processed_stories, f, indent=2)
    
    print(f"\n🎉 Batch processing complete! Data saved to: {all_stories_file}")
    return processed_stories

# Uncomment to process all stories
processed_stories = process_all_stories()


=== Processing Story: The Wise Old Man and the Three Sons ===
API Test - Status Code: 404
⚠️ Unexpected response: Not Found

🎨 Generating image 1/5
Attempting to generate: Golden late-afternoon courtyard: an elderly man in... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: generated_content\images\wisdom-1_scene_01.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 2/5
Attempting to generate: Morning in the paddy: the first son kneeling in sa... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: generated_content\images\wisdom-1_scene_02.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 3/5
Attempting to generate: Bustling bazaar: the second son at a crowded marke... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: generated_content\images\wisdom-1_scene_03.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 4/5
Attempting to generate: Compassionate act: the third son bending to give h...

MoviePy - Done.
Moviepy - Writing video generated_content\videos\wisdom-1_video.mp4



Moviepy - Done !
Moviepy - video ready generated_content\videos\wisdom-1_video.mp4
✅ Successfully processed: The Wise Old Man and the Three Sons

=== Processing Story: The Little Sparrow and the Storm ===
API Test - Status Code: 404
⚠️ Unexpected response: Not Found

🎨 Generating image 1/5
Attempting to generate: Wide majestic banyan: a tiny sparrow perched high ... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: generated_content\images\courage-1_scene_01.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 2/5
Attempting to generate: Ominous sky: thick rolling storm clouds gathering ... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: generated_content\images\courage-1_scene_02.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 3/5
Attempting to generate: Panic and flight: villagers scrambling while large... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: generated_content\images\courage-1_scene

MoviePy - Done.
Moviepy - Writing video generated_content\videos\courage-1_video.mp4



Moviepy - Done !
Moviepy - video ready generated_content\videos\courage-1_video.mp4
✅ Successfully processed: The Little Sparrow and the Storm

=== Processing Story: The Magic Pot of the Poor Woman ===
API Test - Status Code: 404
⚠️ Unexpected response: Not Found

🎨 Generating image 1/5
Attempting to generate: Humble hearth: close interior of a mud-walled hut ... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: generated_content\images\kindness-1_scene_01.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 2/5
Attempting to generate: Knock at dusk: a travel-worn stranger in faded clo... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: generated_content\images\kindness-1_scene_02.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 3/5
Attempting to generate: Last handful shared: extreme close-up of rough han... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: generated_content\images\kindness-1_scene

MoviePy - Done.
Moviepy - Writing video generated_content\videos\kindness-1_video.mp4



Moviepy - Done !
Moviepy - video ready generated_content\videos\kindness-1_video.mp4
✅ Successfully processed: The Magic Pot of the Poor Woman

=== Processing Story: The Honest Merchant's Scale ===
API Test - Status Code: 404
⚠️ Unexpected response: Not Found

🎨 Generating image 1/5
Attempting to generate: Market panorama: a vibrant traditional marketplace... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: generated_content\images\justice-1_scene_01.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 2/5
Attempting to generate: Portrait of integrity: the honest merchant careful... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: generated_content\images\justice-1_scene_02.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 3/5
Attempting to generate: Shadowy hoarders: other merchants stacking and hid... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: generated_content\images\justice-1_scene_03.png

MoviePy - Done.
Moviepy - Writing video generated_content\videos\justice-1_video.mp4



Moviepy - Done !
Moviepy - video ready generated_content\videos\justice-1_video.mp4
✅ Successfully processed: The Honest Merchant's Scale

=== Processing Story: The Student and the Empty Cup ===
API Test - Status Code: 404
⚠️ Unexpected response: Not Found

🎨 Generating image 1/5
Attempting to generate: Courtyard arrival: a confident young student in ne... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
❌ Error 402: {"error":"You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits."}
⏳ Waiting 5 seconds before retry...
Attempting to generate: Courtyard arrival: a confident young student in ne... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 2)
❌ Error 402: {"error":"You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits."}
⏳ Waiting 10 seconds before retry...
Attempting to generate: Courtyard arrival: a confident young student

KeyboardInterrupt: 

## Instructions for Integration

1. **Copy generated files** to your web project's `public/generated_content/` folder
2. **Update story data** in your React app with the generated JSON
3. **Test the video playback** in your web application
4. **Adjust timing and effects** as needed for better user experience

### File Structure for Web App:
```
public/
├── generated_content/
│   ├── images/
│   │   ├── wisdom-1_scene_1.png
│   │   ├── wisdom-1_scene_2.png
│   │   └── ...
│   ├── audio/
│   │   └── wisdom-1_narration.mp3
│   └── videos/
│       └── wisdom-1_video.mp4
```